Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.core_expr_types.tuples</a>
========

In [1]:
import proveit
from proveit import (Lambda, ExprTuple, ExprRange, IndexedVar, safe_dummy_var,
                     defaults, Function, ProofFailure)
from proveit import A, a, b, f, g, i, j, k, l, m, n, x, y
from proveit.logic import Equals, NotEquals, Not, EvaluationError, InSet
from proveit.linear_algebra import TensorProd
from proveit.numbers import (zero, one, two, three, four, e as e_nat, Add, Exp,
                             Integer, Mult, Natural, NaturalPos, Neg, num, subtract)
from proveit.core_expr_types import Len
from proveit.core_expr_types import (
        a_i_to_j, x_1_to_n, y_1_to_n, f_i_to_j, f_i_to_jp1)
from proveit.core_expr_types.tuples import (
        len_of_ranges_with_repeated_indices_from_1, range_from1_len,
        len_of_ranges_with_repeated_indices_from_1, len_of_ranges_with_repeated_indices,
        len_of_empty_range_of_ranges)
from proveit.core_expr_types.tuples import general_range_of_ranges
%begin demonstrations

### Proving the equality of ExprTuples

In [2]:
Equals(ExprTuple(a, b, x, y), ExprTuple(a, b, x, y)).prove()

|- (a, b, x, y) = (a, b, x, y)

In [3]:
ExprTuple(a, b, x, y).deduce_equal_or_not(ExprTuple(x, y))

|- (a, b, x, y) != (x, y)

In [4]:
ExprTuple(a, b, x, y).deduce_equal_or_not(
    ExprTuple(x, b, x, y), assumptions=[NotEquals(a, x)])

{a != x} |- (a, b, x, y) != (x, b, x, y)

In [5]:
defaults.assumptions = [InSet(n, NaturalPos)]

In [6]:
partitioned = x_1_to_n.partition(one, assumptions=[InSet(n, NaturalPos)])

In [7]:
eq = Equals(ExprTuple(*partitioned.lhs.entries, y), 
            ExprTuple(*partitioned.rhs.entries, y))

In [8]:
eq.prove(assumptions=[InSet(n, NaturalPos)])

{n in NaturalPos} |- (x_{1}, x_{2}, ..., x_{n}, y) = (x_{1},x_{2}, x_{3}, ..., x_{n}, y)

In [9]:
shifted = x_1_to_n.shift_equivalence(new_start=zero, assumptions=[InSet(n, NaturalPos)])

In [10]:
eq = Equals(ExprTuple(*shifted.lhs.entries, y), 
            ExprTuple(*shifted.rhs.entries, y))

In [11]:
eq.lhs.inner_expr()[0].partition(one)

{n in NaturalPos} |- (x_{1}, x_{2}, ..., x_{n}, y) =  \\ (x_{1},x_{2}, x_{3}, ..., x_{n}, y)

In [12]:
eq.lhs.deduce_equal(eq.rhs, assumptions=[InSet(n, NaturalPos)])

{n in NaturalPos} |- (x_{1}, x_{2}, ..., x_{n}, y) = (x_{0 + 1}, x_{1 + 1}, ..., x_{(n - 1) + 1}, y)

In [13]:
eq = Equals(ExprTuple(a, y, *partitioned.lhs.entries), 
            ExprTuple(a, y, *partitioned.rhs.entries))

In [14]:
eq.prove(assumptions=[InSet(n, NaturalPos)])

{n in NaturalPos} |- (a, y,x_{1}, x_{2}, ..., x_{n}) = (a, y, x_{1},x_{2}, x_{3}, ..., x_{n})

In [15]:
eq.lhs.deduce_equal(eq.rhs, assumptions=[InSet(n, NaturalPos)])

{n in NaturalPos} |- (a, y,x_{1}, x_{2}, ..., x_{n}) = (a, y, x_{1},x_{2}, x_{3}, ..., x_{n})

### Mapping elements can be useful for building new ExprTuples from existing ones

In [16]:
eq.lhs.map_elements(lambda x : Not(x))

([not](a), [not](y),[not](x_{1}), [not](x_{2}), ..., [not](x_{n}))

In [17]:
ExprTuple.map_elements_together(lambda x, y : Equals(x, y), eq.lhs, eq.lhs)

(a = a, y = y,(x_{1} = x_{1}), (x_{2} = x_{2}), ..., (x_{n} = x_{n}))

**For efficiency, this is limited: ExprRanges must be aligned**

In [18]:
try:
    ExprTuple.map_elements_together(lambda x, y : Equals(x, y), eq.lhs, eq.rhs)
    assert False, "ValueError expected"
except ValueError as e:
    print("Expected error:", e)

Expected error: Each of the ExprTuples must have the same number of entries when mapping elements together


### Testing the `ExprRange(order='decreasing')` flag

An ExprRange with the `order='decreasing'` flag produces an ExprRange of the form f(i),...,f(j), where 0 <= (i-j) and j < i (i.e. the ExprRange represents 0 or more elements and is in decreasing order)

For example, if

self = $f(3),...,f(6)$,

then `order='decreasing'` would return:

$|- f(6),...,f(3)$

In [19]:
test = ExprRange(i, IndexedVar(a, i), j, zero, order='decreasing')

In [20]:
inc_test = ExprRange(i, IndexedVar(a, i), j, zero)

In [21]:
try:
    ExprRange(i, IndexedVar(a, i), two, zero, order='increasing')
    assert False, "Expecting a ValueError; should not get this far!"
except ValueError as _e:
    print("Expected exception: ", _e)

Expected exception:  Invalid ExprRange endpoints: 2 to 0


In [22]:
ExprRange(i, IndexedVar(a, i), two, zero, order='decreasing')

a_{2}, a_{1}, ..., a_{0}

In [23]:
test.lambda_map

_a -> a_{-_a}

In [24]:
(test.start_index, test.end_index, test.true_start_index, test.true_end_index)

(j, 0, -j, 0)

In [25]:
(inc_test.start_index, inc_test.end_index, 
 inc_test.true_start_index, inc_test.true_end_index)

(j, 0, j, 0)

In [26]:
test.body

a_{-_a}

In [27]:
inc_test.body

a_{_a}

In [28]:
test.first()

a_{j}

In [29]:
inc_test.first()

a_{j}

In [30]:
test.last()

a_{0}

In [31]:
inc_test.last()

a_{0}

In [32]:
empty_test_inc = ExprRange(i, IndexedVar(a, i), one, zero, order='increasing')

In [33]:
empty_test_inc.reduction()

|- (a_{1}, a_{2}, ..., a_{0}) = ()

In [34]:
empty_test_dec = ExprRange(i, IndexedVar(a, i), zero, one, order='decreasing')

In [35]:
empty_test_dec.reduction()

|- (a_{0}, a_{-1}, ..., a_{1}) = ()

In [36]:
empty_test_dec.lambda_map

_a -> a_{-_a}

In [37]:
partition_test_inc = ExprRange(i, IndexedVar(a, i), zero, four, order='increasing')

In [38]:
partition_test_inc.partition(two)

|- (a_{0}, a_{1}, ..., a_{4}) =  \\ (a_{0}, a_{1}, ..., a_{2},a_{3}, ..., a_{4})

In [39]:
partition_test_dec = ExprRange(i, IndexedVar(a, i), four, zero, order='decreasing')

In [40]:
partition_test_dec.start_index, partition_test_dec.true_start_index

(4, -4)

In [41]:
partition_test_dec.partition(two)

|- (a_{4}, a_{3}, ..., a_{0}) =  \\ (a_{4}, a_{3}, ..., a_{2},a_{1}, ..., a_{0})

In [42]:
from proveit import fa
from proveit.numbers import Complex

In [43]:
assumptions = [InSet(Add(j, Neg(i), one), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

In [44]:
f_i_to_j_dec = ExprRange(a, fa, i, j, order='decreasing')

In [45]:
from proveit.core_expr_types.tuples import negated_shift_equivalence

In [46]:
negated_shift_equivalence

|- forall_{f} [forall_{a in Integer} [forall_{i, j, k, l | ((j - 1) - i) in Natural, k = (i - a), l = (j - a)} ((f(i), f(i - 1), ..., f(j)) = (f(k + a), f((k - 1) + a), ..., f(l + a)))]]

In [47]:
# negated_shift_equivalence.instantiate({i:Neg(i), j:Neg(j), k:subtract(i, one), a:Neg(one), l:subtract(j, one)}, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
# #                                                                  Natural), InSet(subtract(subtract(j, one), i),
#                                                                  Natural)])

In [48]:
# f_i_to_j_dec.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
#                                                                  Natural)])

In [49]:
# f_i_to_j__shift__eq_dec = \
#     f_i_to_j_dec.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
#                                                                  Natural)]).lhs.entries[0]

In [50]:
# assert False

In [51]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), one, n)).computation(assumptions=[InSet(n, NaturalPos)])

{n in NaturalPos} |- |(a_{1, 1}, a_{1, 2}, ..., a_{1, 0}, a_{2, 1}, a_{2, 2}, ..., a_{2, 0}, ......, a_{n, 1}, a_{n, 2}, ..., a_{n, 0})| = 0

In [52]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), two, n), one, zero)).computation(assumptions=[InSet(n, Natural)])

|- |(a_{1, 2}, a_{1, 3}, ..., a_{1, n}, a_{2, 2}, a_{2, 3}, ..., a_{2, n}, ......, a_{0, 2}, a_{0, 3}, ..., a_{0, n})| = 0

In [53]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, n), one, n)).computation(assumptions=[InSet(n, Natural), InSet(n, NaturalPos)])

{n in NaturalPos, n in Natural} |- |(a_{1, 1}, a_{1, 2}, ..., a_{1, n}, a_{2, 1}, a_{2, 2}, ..., a_{2, n}, ......, a_{n, 1}, a_{n, 2}, ..., a_{n, n})| = n^{2}

In [54]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), two, one)).computation(assumptions=[InSet(n, Natural)])

|- |(a_{2, 1}, a_{2, 2}, ..., a_{2, 0}, a_{3, 1}, a_{3, 2}, ..., a_{3, 0}, ......, a_{1, 1}, a_{1, 2}, ..., a_{1, 0})| = 0

The following 5 cells have been commented out — they create and then depend on the one-element expr_range in the first cell, which returns an error

In [55]:
# expr_range = ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)

In [56]:
# lambda_map = Lambda((expr_range.parameter, expr_range.body.parameter), expr_range.body.body)

In [57]:
# singular_nested_range_reduction.instantiate({f:lambda_map, i:one, j:one, m:one})

In [58]:
# Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)).computation()

In [59]:
# Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, two), one, one)).computation()

In [60]:
assumptions = [InSet(Add(j, Neg(i), one), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

In [61]:
f_i_to_jp1

f(i), f(i + 1), ..., f(j + 1)

In [62]:
eq = f_i_to_jp1.partition(j, assumptions=assumptions)

In [63]:
eq.rhs.merger(assumptions=assumptions)

{(j - i + 1) in Natural} |- (f(i), f(i + 1), ..., f(j), f(j + 1)) = (f(i), f(i + 1), ..., f(j + 1))

In [64]:
f_i_to_j__shift__eq = \
    f_i_to_j.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(Add(j, one), i),
                                                                 Natural)])

In [65]:
f_i_to_j__shift__eq.rhs

(f((i - 1) + 1), f(i + 1), ..., f((j - 1) + 1))

In [66]:
Add(subtract(i, one), one).simplification(assumptions=[InSet(i, Integer)])

{i in Integer} |- ((i - 1) + 1) = i

In [67]:
Add(Add(i, Neg(two)), two).simplification(assumptions=[InSet(i, Integer)])

{i in Integer} |- ((i - 2) + 2) = i

In [68]:
Add(subtract(j, one), one).simplification(assumptions=[InSet(j, Integer)])

{j in Integer} |- ((j - 1) + 1) = j

In [69]:
Add(subtract(j, two), two).simplification(assumptions=[InSet(j, Integer)])

{j in Integer} |- ((j - 2) + 2) = j

In [70]:
f_i_to_j__shift__eq.rhs.entries[0].shift_equivalence(
    old_shift=one, new_shift=two, 
    assumptions=[InSet(j, Integer), InSet(i, Integer),
                 InSet(subtract(j, subtract(i, one)), Natural)])

{j in Integer, i in Integer, (j - (i - 1)) in Natural} |- (f((i - 1) + 1), f(i + 1), ..., f((j - 1) + 1)) = (f((i - 2) + 2), f((i - 1) + 2), ..., f((j - 2) + 2))

In [71]:
from proveit import fa
from proveit.numbers import Complex

In [72]:
f_i_to_j_dec = ExprRange(a, fa, i, j, order='decreasing')

In [73]:
assumptions = [InSet(Add(Neg(j), i, Neg(one)), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

In [74]:
#InSet(Neg(Add(x, one)), Complex).prove(assumptions=[InSet(x, Complex)])

In [75]:
Neg(Add(Neg(Add(x, one)), one)).distribution(assumptions=[InSet(x, Complex)])

{x in Complex} |- (-(-(x + 1) + 1)) = x

In [76]:
InSet(subtract(subtract(Neg(j), one), Neg(i)), Natural)

((-j - 1) - (-i)) in Natural

In [77]:
# f_i_to_j__shift__eq_dec = \
#     f_i_to_j_dec.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
#                                                                  Natural)]).lhs.entries[0]

In [78]:
# lhs_done = f_i_to_j__shift__eq_dec.inner_expr().lhs[0].with_decreasing_order()

In [79]:
# lhs_done.inner_expr().rhs[0].with_decreasing_order()

In [80]:
# f_i_to_j__shift__eq_dec.rhs.entries[0].shift_equivalence(
#     old_shift=one, new_shift=two, 
#     assumptions=[InSet(j, Integer), InSet(i, Integer),
#                  InSet(subtract(j, subtract(i, one)), Natural)])

In [81]:
tup=ExprTuple(a, b)

In [82]:
Len(a_i_to_j).computation(assumptions=[InSet(Add(j, Neg(i), one), Natural)])

{(j - i + 1) in Natural} |- |(a_{i}, a_{i + 1}, ..., a_{j})| = (j - i + 1)

In [83]:
tup_jdgmt = tup.prove(assumptions=[tup])

Not clear what the following cell is/was trying to accomplish:

In [84]:
# tup.prove(assumptions=[tup]).instantiate({a:a_i_to_j},
#                             assumptions=[InSet(Add(j, Neg(i), one), Natural)])

In [85]:
eq = f_i_to_jp1.partition(k, assumptions=[InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

In [86]:
eq.rhs.merger(assumptions=[InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

{((k + 1) - i) in Natural, ((j + 1) - k) in Natural} |- (f(i), f(i + 1), ..., f(k),f(k + 1), f(k + 2), ..., f(j + 1)) =  \\ (f(i), f(i + 1), ..., f(j + 1))

In [87]:
eq = f_i_to_jp1.partition(i, assumptions=[InSet(subtract(Add(j, one), i), Natural)])

In [88]:
eq.rhs.merger(assumptions=[InSet(subtract(Add(j, one), i), Natural)])

{((j + 1) - i) in Natural} |- (f(i),f(i + 1), f(i + 2), ..., f(j + 1)) =  \\ (f(i), f(i + 1), ..., f(j + 1))

In [89]:
eq = f_i_to_j.partition(subtract(j, one), assumptions=[InSet(subtract(j, i), Natural)])

In [90]:
Add(subtract(j, one), one).simplification(assumptions=[InSet(j, Natural)])

{j in Natural} |- ((j - 1) + 1) = j

In [91]:
eq.rhs.merger(assumptions=[InSet(subtract(j, i), Natural), InSet(j, Natural)])

{(j - i) in Natural} |- (f(i), f(i + 1), ..., f(j - 1), f(j)) =  \\ (f(i), f(i + 1), ..., f(j))

### Testing the `ExprTuple.range_expansion()` method

For self an ExprTuple with a single entry that is an ExprRange of the form f(i),...,f(j), where 0 <= (j-i) <= 9 (i.e. the ExprRange represents 1 to 10 elements), derive and return an equality between self and an ExprTuple with explicit entries replacing the ExprRange. For example, if

self = ExprTuple$(f(3),...,f(6))$,

then `self.range_expansion()` would return:

$|- (f(3),...,f(6)) = (f(3), f(4), f(5), f(6))$

First not that we cannot create an ExprRange with only one element explicitly:

In [92]:
try:
    ExprTuple(ExprRange(a, Function(g, a), two, two))
    assert False, "Expecting a ValueError"
except ValueError as e:
    print("Expected Error:", e)

Expected Error: Do not create an ExprRange with the same start and end index since it reduces to a single instance.  Note that nested_range/var_range automatically perform this reduction.


We can, however, create an ExprRange that doesn't obviously have one element but may turn out to have one element after a simplification or substitution.

In [93]:
expr_tuple_with_1_elem_range = ExprTuple(ExprRange(a, Function(g, a), two, Mult(one, two)))

In [94]:
expr_tuple_with_1_elem_range.simplification()

|- (g(2), g(3), ..., g(1 * 2)) = (g(2))

ExprRange indices are already expected to be simplified w.r.t. `quick_simplified_index` from `proveit.numbers`, however, which deals with basic `Add` and `Neg` simplifications and combines all literal integers at the end of an `Add`.

In [95]:
try:
    # A one-element ExprRange (hidden by choosing the start and end indices
    # that are equal but appear different)
    example_expr_range_2 = ExprRange(i, IndexedVar(x, i), one, Add(one, zero))
    assert False, "Expecting ValueError"
except ValueError as e:
    print("Expected Error:", e)

Expected Error: Start and end indices must be in the 'quick_simplified_index' canonical form: (1, 1 + 0) ≠ (1, 1)


`range_expansion` does the same thing as `simplification` in this case:

In [96]:
expr_tuple_with_1_elem_range.range_expansion()

|- (g(2), g(3), ..., g(1 * 2)) = (g(2))

In [97]:
# generate some example ExprTuples each containing a single ExprRange element
expr_tuple_with_2_elem_range, expr_tuple_with_2_elem_range_exp, expr_tuple_with_3_elem_range, expr_tuple_with_unknown_elem_range, expr_tuple_too_big = [
        ExprTuple(ExprRange(a, Function(g, a), two, three)),
        ExprTuple(ExprRange(a, Exp(two, Add(a, one)), two, three)),
        ExprTuple(ExprRange(a, Exp(e_nat, Add(a, j)), two, four)),
        ExprTuple(ExprRange(a, Exp(e_nat, Add(a, j)), i, k)),
        ExprTuple(ExprRange(a, Function(g, a), two, num(20)))]

In [98]:
# find equivalent explicitly 2-element ExprTuple
expr_tuple_with_2_elem_range.range_expansion()

|- (g(2), ..., g(3)) = (g(2), g(3))

In [99]:
# find equivalent explicitly 2-element ExprTuple
# notice the automatic simplification
expr_tuple_with_2_elem_range_exp.range_expansion()

|- (2^{2 + 1}, ..., 2^{3 + 1}) = (8, 16)

In [100]:
# find equivalent explicitly 2-element ExprTuple
# notice the automatic simplification
expr_tuple_with_3_elem_range.range_expansion()

|- (e^{2 + j}, e^{3 + j}, ..., e^{4 + j}) = (e^{2 + j}, e^{3 + j}, e^{4 + j})

In [101]:
# When the number of elements represented by the ExprRange is not known,
# the range_expansion() method fails with an informative error message
try:
    expr_tuple_with_unknown_elem_range.range_expansion()
    assert False, "Expecting an EvaluationError; should not get this far!"
except EvaluationError as the_error:
    print("EvaluationError: {}".format(the_error))

EvaluationError: Evaluation of k - i under assumptions {n in NaturalPos} is not known. The ExprRange e^{i + j}, e^{(i + 1) + j}, ..., e^{k + j} must represent a known, finite number of elements, but all we know is that it represents k - i elements.
EvaluationError: Evaluation of k - i under assumptions {n in NaturalPos} is not known


In [102]:
# but if we give the method enough information about the indexing
# we can generate the equivalent explicit ExprTuple
expr_tuple_with_unknown_elem_range.range_expansion(
        assumptions=[Equals(k, Add(i, four)), Equals(subtract(k, i), four)])

{k = (i + 4)} |- (e^{i + j}, e^{(i + 1) + j}, ..., e^{k + j}) = (e^{i + j}, e^{(i + 1) + j}, e^{(i + 2) + j}, e^{(i + 3) + j}, e^{(i + 4) + j})

In [103]:
# When the number of elements represented by the ExprRange is > 10,
# the range_expansion() method fails with an informative error message
try:
    expr_tuple_too_big.range_expansion(assumptions=[Equals(subtract(num(20), two), num(18))])
    assert False, "Expecting an EvaluationError; should not get this far!"
except ValueError as the_error:
    print("ValueError: {}".format(the_error))

ValueError: ExprTuple.range_expansion() implemented only for ExprTuples with a single entry, with the single entry being an ExprRange representing a finite number of elements n with 1 <= n <= 9. Instead, the ExprTuple is g(2), g(3), ..., g(20) with number of elements equal to 19.


In [104]:
# a related issue to ask about
try:
    Equals(subtract(num(20), two), num(18)).prove()
except ProofFailure as the_error:
    print("ProofFailure: {}".format(the_error))

ProofFailure: Unable to prove (20 - 2) = 18 assuming {n in NaturalPos}:
'conclude' method not implemented for proof automation


A related example where the ExprRange and ExprTuple are being used as an argument for a TensorProd operator.

In [105]:
# create an example ExprRange
example_expr_range = ExprRange(i, IndexedVar(x, i), one, three)

In [106]:
# use the example ExprRange as the arg(s) for a TensorProd:
example_tensor_prod_over_range = TensorProd(example_expr_range)

In [107]:
# find the ExprTuple equivalent to the (ExprTuple-wrapped) ExprRange:
expr_range_eq_expr_tuple = ExprTuple(example_expr_range).range_expansion()

In [108]:
# find the equivalent explicit tensor prod
example_tensor_prod_over_range.inner_expr().operands.substitution(
         expr_range_eq_expr_tuple)

|- (x_{1} otimes  x_{2} otimes  ... otimes  x_{3}) =  \\ (x_{1} otimes x_{2} otimes x_{3})

In [109]:
%end demonstrations